In [2]:
%pip install folium pandas shapely geopandas

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------ --------------- 1.0/1.7 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 4.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   -- ------------------------------------- 1.0/19.2 MB 5.7 MB/s eta 0:00:04
   ---- ----------------------------------- 2.1/19.2 MB 5.6 MB/s eta 0:00:04
   ------- -------------------------------- 3.4/19.2 MB 5.7 MB/s eta 0:00:03
   --------- ------------------------------ 4.7/19.2 MB 5.8 MB/s eta 0:00:03
   ------------ --------------------------- 6.0/19.2 MB 5.9 MB/s eta 0:00:03
   -------------- ------------------------- 7.1/19.2 MB 5.8 MB/s eta 0:00:03
   ----------------- ---------------------- 8.4/19.2 MB 5.9 MB/s eta 0:00:02
   ------------------- -------------------- 9.4/19.2 MB 5.9 MB/s eta 0:00:02
   ---------------------- ----------------- 10.7/19.2 MB 5.9 MB/s eta 0:00:02
   ------------

In [1]:
import folium
import pandas as pd
import json

### **selected_area.geojson 파일 이동(select_map.html에서 polygon export한 뒤에 실행)**

In [16]:
# 다운로드 된 selected_area.geojson 파일 위치 변경
import os
import shutil

# 현재 노트북이 실행 중인 폴더 (루트 폴더)
root_dir = os.getcwd()

# 브라우저 기본 다운로드 폴더
download_dir = os.path.expanduser("~/Downloads")

# 원본 파일 경로
src_file = os.path.join(download_dir, "selected_area.geojson")

# 이동할 경로 (루트 폴더 안)
dst_file = os.path.join(root_dir, "selected_area.geojson")

# 파일이 존재할 때만 이동
if os.path.exists(src_file):
    shutil.move(src_file, dst_file)
    print(f"✅ 파일 이동 완료: {dst_file}")
else:
    print("❌ 다운로드 폴더에서 selected_area.geojson을 찾을 수 없습니다.")

✅ 파일 이동 완료: C:\Users\rlatj\Programming_Workspace\Hanyang_Programming\Soomgil\data\polygon\selected_area.geojson


### **polygon 저장(target_tag 설정 필수)**

In [2]:
target_tag = 'mountain' # park, river, mountain  *********************실행 전 확인************************

# properties의 type 설정
with open("selected_area.geojson") as f:
    geojson_data = json.load(f)

# # polygon_data 폴더에 잘 저장됐는지 확인할 때 위의 코드 대신에 실행
# with open("polygon_data/park_area.geojson") as f:
#     geojson_data = json.load(f)

for i, feature in enumerate(geojson_data["features"]):
    feature["properties"]["type"] = target_tag

FileNotFoundError: [Errno 2] No such file or directory: 'selected_area.geojson'

### **지표에 polygon 표시(확인용)**

In [18]:
nodes_gdf = pd.read_csv("../raw/nodes.csv")

center_lat = nodes_gdf["lat"].mean()
center_lon = nodes_gdf["lon"].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=14)

# Esri.WorldImagery 스타일 적용
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Tiles © Esri — Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, '
         'Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community',
    name='Esri.WorldImagery',
    overlay=False,
    control=True
).add_to(m)

# polygon 표시
folium.GeoJson(
    geojson_data,
    name="polygons",
    tooltip=folium.GeoJsonTooltip(fields=["type"], aliases=["Type"])
).add_to(m)
m

### **selected_area.geojson 파일 백업**

In [19]:
# 백업 폴더 경로
backup_dir = os.path.join(root_dir, "polygon_data")

# 백업 파일명 생성
backup_file = os.path.join(
    backup_dir, f"{target_tag}_area.geojson"
)

# geojson_data를 polygon_data에 저장
with open(backup_file, "w", encoding="utf-8") as f:
    json.dump(geojson_data, f, ensure_ascii=False, indent=2)
    
print(f"💾 geojson_data를 백업했습니다: {backup_file}")

💾 geojson_data를 백업했습니다: C:\Users\rlatj\Programming_Workspace\Hanyang_Programming\Soomgil\data\polygon\polygon_data\mountain_area.geojson


### **polygon 추가하기(필요에 따라 실행)**

In [21]:
file_path = "polygon_data/park_area.geojson" # *********************실행 전 확인************************

# 기존 데이터 불러오기
with open(file_path, encoding="utf-8") as f:
    original_data = json.load(f)

# Feature 합치기
merged = {
    "type": "FeatureCollection",
    "features": original_data["features"] + geojson_data["features"]
}

# 저장하기
with open("polygon_data/park_area_added.geojson", "w", encoding="utf-8") as f: # *********************실행 전 확인************************
    json.dump(merged, f, ensure_ascii=False, indent=2)